## Sparrow Demo

### 1. Start an instance

Envirement requirements:
- docker
- git
- github

In [1]:
import sys
sys.path.append("..")

import utils
import time

# bootstrap
bootstrap = utils.Bootstrap("../worksapce.config.json")
bootstrap.run()

biz_tags = [{"Key": "id", "Value": "rec-demo"}]

In [2]:
aliyun_ecs_manager = utils.AliyunECSManager()
cmd_generator = utils.CommandGenerator()

aliyun_ecs_manager.clean(biz_tags)

settings = aliyun_ecs_manager.create_default_settings()
settings.set_biz_tags(biz_tags)
settings.set_instance_type("ecs.g6.xlarge")
vm_property = aliyun_ecs_manager.create_instance(settings)

time.sleep(20)

cmd_content = "\n".join([
    cmd_generator.generate_install_docker_command(),
    cmd_generator.generate_install_git_command(),
    cmd_generator.generate_config_github_command(),
    # cmd_generator.generate_start_jupyter_command()
])

aliyun_ecs_manager.execute_command(instance_id=vm_property.instance_id,
                                  cmd_content=cmd_content)

't-sgp4cbl4gila800'

In [3]:
vm_properties = aliyun_ecs_manager.get_instances(biz_tags=biz_tags)

login_cmd = """
ssh -o UserKnownHostsFile=/dev/null root@{server}
""".format(server=vm_properties[0].public_ip)

print(login_cmd)


ssh -o UserKnownHostsFile=/dev/null root@47.236.229.233



### 2. login the instance

### 3. Clone code
```bash
mkdir code
cd code
git clone git@github.com:dream-365/SparrowRecSys.git
```

In [4]:
clone_code_cmd = """
mkdir work
cd work
git clone git@github.com:dream-365/SparrowRecSys.git
"""

aliyun_ecs_manager.execute_command(instance_id=vm_property.instance_id,
                                  cmd_content=clone_code_cmd)

't-sgp4cblf09h2lts'

### 4. Compile prject
```bash
cd SparrowRecSys
docker run -it --rm --name sparrow-maven-project \
-v "$(pwd)":/usr/src/sparrow \
-w /usr/src/sparrow maven:3.3-jdk-8 \
mvn package
```

In [6]:
compile_code_cmd = """
cd work/SparrowRecSys
docker run -it --rm --name sparrow-maven-project \
-v "$(pwd)":/usr/src/sparrow \
-w /usr/src/sparrow maven:3.3-jdk-8 \
mvn package
"""

aliyun_ecs_manager.execute_command(instance_id=vm_property.instance_id,
                                  cmd_content=compile_code_cmd)

't-sgp4cbm1qjbmqrk'

### 5. Run project

```bash
docker run --rm \
-v "$PWD":/usr/src/sparrow \
-p 9010:6010 \
-w /usr/src/sparrow openjdk:11 \
java -cp target/SparrowRecSys-1.0-SNAPSHOT-jar-with-dependencies.jar com.sparrowrecsys.online.RecSysServer
```

In [7]:
run_project_cmd = """
cd work/SparrowRecSys
docker run -d --rm \
-v "$PWD":/usr/src/sparrow \
-p 9010:6010 \
-w /usr/src/sparrow openjdk:11 \
java -cp target/SparrowRecSys-1.0-SNAPSHOT-jar-with-dependencies.jar com.sparrowrecsys.online.RecSysServer
"""

aliyun_ecs_manager.execute_command(instance_id=vm_property.instance_id,
                                  cmd_content=run_project_cmd)

't-sgp4cbm8j1vt2bk'

### 7. Get recommendation

e.g:
```
http://47.236.229.233:9010/getrecforyou?id=427&size=32&model=emb
```

### 6. Clenup 

In [ ]:
aliyun_ecs_manager = utils.AliyunECSManager()
aliyun_ecs_manager.clean(biz_tags)